# Houston, TX Simulations
8760 rows per simulation<br>
50 files (removed cell-in-cell)

In [ ]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [ ]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 1.11
elec = 0.112

---

## Combine all csv files in directory

In [ ]:
# Create variable for files in directory
files = [f for f in os.listdir("data/hou/") if f.endswith(".csv")]

#files

In [ ]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/hou/" + file)
    all_data = pd.concat([all_data, df])
    
all_data

# 438000  rows, 73 columns

In [ ]:
all_data.dtypes

In [ ]:
# print(all_data)
# is_NaN = all_data. isnull()
# row_has_NaN = is_NaN. any(axis=1)
# rows_with_NaN = all_data[row_has_NaN]
# print(rows_with_NaN)

In [ ]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) "]]

#all_data1

In [ ]:
all_data1 = all_data1.dropna().reset_index(drop=True)

# all_data1

#438000  rows, 73 columns

-----

## Get just the wanted data (columns)

In [ ]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data2

#438000  rows, 6 columns

-----

## Get annual energy data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [ ]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [ ]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [ ]:
# Drop columns
get_annual.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual

---

## Get annual cost data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>
Add cost multiplier, variable needs to be defined above

In [ ]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [ ]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanEnergy($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingEnergy($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingEnergy($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [ ]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs

---

## Combine annual energy and cost data
Combine two dfs<br>
Group by scenario <br>
Export combine file

In [ ]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs, how="inner" )

#allAnnualData

In [ ]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanEnergy($)"] + allAnnualData["HeatingEnergy($)"] + allAnnualData["CoolingEnergy($)"]
allAnnualData["AnnualCost($)"] = totalCost

allAnnualData

---
## Get scenario names 
Get the names to a list<br>
Export to file

In [ ]:
# # Create new df of just scenario names
# # Export as a csv for javascript filter
# scenarios = allAnnualData["Scenario"].unique()
# print(scenarios)

In [ ]:
# scenarios.tofile("data/final/hou_scenarios.csv", sep=",", format="%s")

---
## Group and export
Group dataframe by scenario<br>
Export to file

In [ ]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario"])

annualData.sum().round(2)

In [ ]:
# Export grouped file to csv
annualData.sum().round(2).to_csv("../Scraper_Output/hou_annual_data.csv", header=True, index=True)

-----

## Get daily and monthly energy data
Group by scenario <br>
Groups by month <br>
Change units from Joules to therms and kWh

In [ ]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [ ]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [ ]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_monthly

In [ ]:
# get_monthly["Month"] = get_monthly["Date_Time"].apply(lambda x: str(x)[0:3])

# get_monthly

## Get monthly cost data

Group by scenario <br>
Groups by month <br>
Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [ ]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [ ]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanEnergy($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingEnergy($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingEnergy($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [ ]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_monthly_costs

In [ ]:
# get_monthly_costs["Month"] = get_monthly_costs["Date_Time"].apply(lambda x: str(x)[0:3])

# get_monthly_costs

---

## Combine monthly energy and cost data
Combine two dfs<br>
Create Month Column<br>
Group by scenario <br>
Export combine file

In [ ]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner" )

#allMonthlyData

In [ ]:
# Add energy and cost totals
totalEnergyMonthly = allMonthlyData["FanEnergy(kWh)"] + allMonthlyData["HeatingEnergy(therms)"] + allMonthlyData["CoolingEnergy(kWh)"]
allMonthlyData["MonthlyEnergy(kBtu)"] = totalEnergyMonthly

totalCostMonthly = allMonthlyData["FanEnergy($)"] + allMonthlyData["HeatingEnergy($)"] + allMonthlyData["CoolingEnergy($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

averageIlluminanceMonthly = allMonthlyData["Illuminance(lux)"].mean()

#allMonthlyData

In [ ]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

allMonthlyData

In [ ]:
# Drop Illuminance column...don't need the totals
allMonthlyData2 = allMonthlyData.drop(["Illuminance(lux)", "MonthlyEnergy(kBtu)" ], axis=1)
allMonthlyData2

In [ ]:
# Group by scenario and date/time for export to graphing
MonthlyData = allMonthlyData2.groupby(["Scenario", "Date_Time"])

MonthlyData.sum()

In [ ]:
# Export grouped file to csv (used before grouped by month)
MonthlyData.sum().round(2).to_csv("../Scraper_Output/hou_monthly_data.csv", header=True, index=True)

In [ ]:
# # Export grouped file to json, issues with exporting grouped csv
# MonthlyData.sum().round(2).to_json("data/final/hou_monthly_data.json")

In [ ]:
# Group by scenario and date/time for export to graphing
MonthlyData2 = allMonthlyData2.groupby(["Scenario", "Month"])

MonthlyData2.sum()

In [ ]:
# Export grouped file to csv
MonthlyData2.sum().round(2).to_csv("../Scraper_Output/hou_monthly2_data.csv", header=True, index=True)

## Get monthly average illuminance
Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [ ]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]
Illuminance

In [ ]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Date_Time"])

MonthlyIlluminance.mean()

In [ ]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("../Scraper_Output/hou_monthly_illuminance.csv", header=True, index=True)

In [ ]:
# # Export grouped file to json, issues with exporting grouped csv
# MonthlyIlluminance.mean().round(2).to_json("data/final/hou_monthly_illuminance.json")

In [ ]:
# Group by scenario
MonthlyIlluminance2 = Illuminance.groupby(["Scenario", "Month"])

MonthlyIlluminance2.mean()

In [ ]:
# Export grouped file to csv
MonthlyIlluminance2.mean().round(2).to_csv("../Scraper_Output/hou_monthly_illuminance2.csv", header=True, index=True)

------

## Get hourly data

In [ ]:
all_data3 = all_data1.copy(deep=True)

#all_data3

In [ ]:
# Get wanted columns
# Include hourly illumanc
all_data3 = all_data3[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) "]]

#all_data3

In [ ]:
# Rename Columns
get_hourly = all_data3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance[lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly

In [ ]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly["FanEnergy($)"] = (get_hourly["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly["HeatingEnergy($)"] = ((get_hourly["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly["CoolingEnergy($)"] = (get_hourly["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_hourly

In [ ]:
# Drop columns
get_hourly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)


------

## Get hourly cost data

In [ ]:
get_hourly_costs = all_data3.copy(deep=True)

#get_hourly_costs

In [ ]:
# Rename Columns
get_hourly_costs = get_hourly_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance[lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly_costs

In [ ]:
# Drop columns
get_hourly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)


## Combine hourly energy and cost data

In [ ]:
# Combine data for energy and cost
allHourlyData = pd.merge(get_hourly, get_hourly_costs, how="inner" )

allHourlyData

In [ ]:
# Export grouped file to csv for daily cost sums
allHourlyData.to_csv("../Scraper_Output/hou_daily_cost_sum.csv")